In [40]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from qiskit import QuantumCircuit, transpile, Aer, execute, assemble
from qiskit.quantum_info import state_fidelity
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram

In [41]:
import pandas as pd
import numpy as np
df = pd.read_csv('bank_loan.csv')

In [42]:
df['CODE_GENDER'] = df['CODE_GENDER'].map({'M': 1, 'F': 0})
df['FLAG_OWN_CAR'] = df['FLAG_OWN_CAR'].map({'Y': 1, 'N': 0})
df['FLAG_OWN_REALTY'] = df['FLAG_OWN_REALTY'].map({'Y': 1, 'N': 0})
df['NAME_CONTRACT_TYPE'] = df['NAME_CONTRACT_TYPE'].map({'Cash loans': 1, 'Revolving loans': 0})
df['NAME_TYPE_SUITE_INT'] = pd.factorize(df['NAME_TYPE_SUITE'])[0]
df['NAME_INCOME_TYPE_INT'] = pd.factorize(df['NAME_INCOME_TYPE'])[0]
df['NAME_FAMILY_STATUS_INT'] = pd.factorize(df['NAME_FAMILY_STATUS'])[0]
df['OCCUPATION_TYPE_INT'] = pd.factorize(df['OCCUPATION_TYPE'])[0]

df.to_csv("normalized_dataset.csv", index=False)

In [43]:
data = pd.read_csv("normalized_dataset.csv")
data

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_TYPE_SUITE_INT,NAME_INCOME_TYPE_INT,NAME_FAMILY_STATUS_INT,OCCUPATION_TYPE_INT
0,100002,1,1,1.0,0,1,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0
1,100003,0,1,0.0,0,0,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1
2,100004,0,0,1.0,1,1,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
3,100006,0,1,0.0,0,1,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2,0
4,100007,0,1,1.0,0,1,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,1,1.0,0,0,0,157500.0,254700.0,27558.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,4,5
307507,456252,0,1,0.0,0,1,0,72000.0,269550.0,12001.5,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3,3,-1
307508,456253,0,1,0.0,0,1,0,153000.0,677664.0,29979.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0,0,4,3
307509,456254,1,1,0.0,0,1,0,171000.0,370107.0,20205.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,2,1,0


In [44]:
unwanted=[
'NAME_TYPE_SUITE',
'NAME_INCOME_TYPE',
'NAME_EDUCATION_TYPE',
'NAME_FAMILY_STATUS',
'NAME_HOUSING_TYPE',
'OCCUPATION_TYPE',
'WEEKDAY_APPR_PROCESS_START',
'ORGANIZATION_TYPE',
'FONDKAPREMONT_MODE',
'HOUSETYPE_MODE',
'WALLSMATERIAL_MODE',
'EMERGENCYSTATE_MODE']
data.drop(labels=unwanted,axis=1,inplace=True)

In [45]:
columns_with_nan = data.columns[data.isna().any()].tolist()

data[columns_with_nan] = data[columns_with_nan].fillna(method='ffill')

data[columns_with_nan] = data[columns_with_nan].fillna(method='bfill')

In [46]:
X = data.drop('TARGET', axis=1)
y = data['TARGET']

In [47]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [48]:
scaler=StandardScaler()
scaler.fit(X)

StandardScaler()

In [49]:
scaled_data=scaler.transform(X)

In [50]:
pca=PCA(n_components=0.50)
pca.fit(scaled_data)
x_pca=pca.transform(scaled_data)

In [51]:
feature_values = x_pca[0]  # Use the first row of PCA-transformed data as feature values
num_qubits = len(feature_values)

In [52]:
from qiskit import Aer, execute, QuantumCircuit
from qiskit.quantum_info import Statevector
import numpy as np

def feature_map_2(num_qubits, feature_values):
    qc = QuantumCircuit(num_qubits)
    
    # Apply Hadamard gates to each qubit
    for i in range(num_qubits):
        qc.h(i)

    # Encode feature values using RZ-gates
    for i, feature_value in enumerate(feature_values):
        rz_angle = 2 * np.pi * feature_value
        qc.rz(rz_angle, i)

    # Implement RZZ-gates for second order data encoding
    for i in range(num_qubits - 1):
        for j in range(i + 1, num_qubits):
            qc.cx(i, j)
            rz_angle = np.pi - feature_values[i] - feature_values[j]
            qc.rz(rz_angle, j)
            qc.cx(i, j)

    # Repeat the feature map to create a depth of 2
    qc.compose(qc, inplace=True)
    
    return qc

In [53]:
def calculate_fidelity(feature_map_function, num_qubits, feature_values):
    # Create the quantum circuit using the provided feature_map function
    qc = feature_map_function(num_qubits, feature_values)
    
    # Define the statevector simulator backend
    simulator = Aer.get_backend('statevector_simulator')
    
    # Execute the circuit on the statevector simulator
    job = execute(qc, simulator)
    result = job.result()
    
    # Get the statevector of the output state
    output_state = result.get_statevector(qc)
    
    # Define the reference state (target state)
    reference_qc = feature_map_function(num_qubits, feature_values)
    reference_state = Statevector.from_instruction(reference_qc)
    
    # Calculate the fidelity between the output state and the reference state
    fidelity = np.abs(np.dot(output_state.data.conj(), reference_state.data)) ** 2
    
    return fidelity


In [54]:
fidelity = calculate_fidelity(feature_map_2, num_qubits, feature_values)
print(f"Quantum fidelity: {fidelity}")

Quantum fidelity: 0.9999999999999969


In [55]:
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Select records from index 10 to 20
selected_records = x_pca[10:21]  # Assuming x_pca is your PCA-transformed data

# Iterate over the selected records and compute fidelity
fidelity_values = []
for record in selected_records:
    feature_values = record
    num_qubits = len(feature_values)
    fidelity = calculate_fidelity(feature_map_2, num_qubits, feature_values)
    fidelity_values.append(fidelity)

# Plot the fidelity values with a zoomed-in y-axis (line plot)
plt.figure(figsize=(10, 5))
plt.plot(range(10, 21), fidelity_values, marker='o')
plt.xlabel('Record Index')
plt.ylabel('Fidelity')
plt.title('Fidelity for Records 10 to 20')
plt.ylim([min(fidelity_values) * 0.99, max(fidelity_values) * 1.01])  # Zoom in on y-axis
plt.grid(True)
plt.savefig('fidelity_line_plot(new).svg', format='svg')
plt.show()


In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


In [57]:
model = Sequential()


model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))


model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=4, activation='relu'))
model.add(Dense(units=2, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [58]:
# Function to extract activations from the 2nd last hidden layer
def get_activations(model, layer_index, X):
    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=model.layers[layer_index].output)
    activations = intermediate_layer_model.predict(X)
    return activations
layer_index = -2
activations = get_activations(model, layer_index, X_train)

In [59]:
def quantum(sample_idx_to_print, activations):
    num_qubits = activations.shape[1]
    qc1 = QuantumCircuit(num_qubits, num_qubits)  # Include classical bits for measurements
    
    # Apply Hadamard gates to all qubits
    qc1.h(range(num_qubits))
    
    # Apply controlled-Z gates to create entanglement
    for i in range(num_qubits - 1):
        for j in range(i + 1, num_qubits):
            qc1.cz(i, j)
    
    # Apply parameterized single-qubit rotations based on activations
    for i in range(num_qubits):
        angle = activations[sample_idx_to_print, i]
        qc1.rx(angle, i)  # Use Rx gate instead of U gate for simplicity
        
    qc1.barrier()  # Add a barrier before measurements (optional)
    
    # Measure all qubits
    qc1.measure(range(num_qubits), range(num_qubits))
    
    return qc1

In [60]:
# Function to calculate fidelity, density matrix, and depth of the quantum circuit
def analyze_quantum_circuit(activations, sample_idx):
    qc1 = quantum(sample_idx, activations)
    
    # Statevector simulator for fidelity calculation and density matrix
    backend_sv = Aer.get_backend('statevector_simulator')
    result_sv = execute(qc1, backend_sv).result()
    statevector = result_sv.get_statevector(qc1)
    density_matrix = np.outer(statevector, np.conj(statevector))
    
    # QASM simulator for measurements
    backend_qasm = Aer.get_backend('qasm_simulator')
    transpiled_qc = transpile(qc1, backend_qasm)
    qobj = assemble(transpiled_qc, shots=1024)
    result_qasm = backend_qasm.run(qobj).result()
    counts = result_qasm.get_counts()
    
    # Calculate circuit depth
    backend_unitary = Aer.get_backend('unitary_simulator')
    transpiled_qc = transpile(qc1, backend_unitary)
    depth = transpiled_qc.depth()
    
    # Plot histogram using matplotlib
    plt.bar(counts.keys(), counts.values())
    plt.xlabel('Outcome')
    plt.ylabel('Frequency')
    plt.title('Measurement Outcomes')
    plt.show()
    
    return statevector, density_matrix, counts, depth

# Calculate fidelity, density matrix, and generate histogram for a specific sample
sample_idx = 0  # Choose a sample index
statevector, density_matrix, counts, depth = analyze_quantum_circuit(activations, sample_idx)

print(f"Circuit Depth: {depth}")

Circuit Depth: 8


In [61]:
# Plot histogram using Matplotlib with adjusted bar width
plt.figure(figsize=(8, 6))
plt.bar(counts.keys(), counts.values(), width=0.8)  # Adjust the width parameter as needed
plt.xlabel('Outcome')
plt.ylabel('Frequency')
plt.title(f'Measurement Outcomes for record {sample_idx}')
plt.savefig('Measurement_Count_Histogram.svg', format='svg')
plt.show()


In [62]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Check the size of the density matrix
matrix_size = density_matrix.shape[0]

# Determine the number of qubits based on the size of the density matrix
num_qubits = int(np.log2(matrix_size))

# Reshape the density matrix to a square matrix
density_matrix_square = density_matrix.reshape((2 ** num_qubits, 2 ** num_qubits))

# Plot the real part of the density matrix
plt.figure(figsize=(10, 8))
plt.title('Real Part of Density Matrix')
sns.heatmap(np.real(density_matrix_square), annot=True, fmt=".2f", cmap='viridis')
plt.xlabel('Qubit Index')
plt.ylabel('Qubit Index')
#plt.savefig('real_density_matrix.png')  # Save the figure as an image
plt.savefig('real_density_matrix.svg', format='svg')

plt.show()

# Plot the imaginary part of the density matrix
plt.figure(figsize=(10, 8))
plt.title('Imaginary Part of Density Matrix')
sns.heatmap(np.imag(density_matrix_square), annot=True, fmt=".2f", cmap='viridis')
plt.xlabel('Qubit Index')
plt.ylabel('Qubit Index')
#plt.savefig('imaginary_density_matrix.png')  # Save the figure as an image
plt.savefig('imaginary_density_matrix.svg', format='svg')

plt.show()


In [63]:
# Function to calculate depth for a group of samples
def calculate_depths_for_group(activations, start_idx, end_idx):
    depths = []
    for sample_idx in range(start_idx, end_idx):
        _, _, _, depth = analyze_quantum_circuit(activations, sample_idx)
        depths.append(depth)
    return depths

# Function to calculate the moving average
def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size) / window_size, mode='valid')

# Assuming activations contains the quantum circuit activations for all samples

# Extract depths for the first 100 samples
depth_values = calculate_depths_for_group(activations, 0, 100)

# Smooth the depth values using a moving average
window_size = 10  # Adjust the window size to get a smoother curve
smoothed_depth_values = moving_average(depth_values, window_size)

# Calculate the average circuit depth
average_depth = np.mean(depth_values)

# Plot the depth values for the first 100 samples
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(smoothed_depth_values) + 1), smoothed_depth_values, marker='o', linestyle='-', color='green', label='Smoothed Depth')
plt.axhline(average_depth, color='red', linestyle='--', label=f'Average Depth: {average_depth:.2f}')
plt.xlabel('Record Index')
plt.ylabel('Circuit Depth')
plt.title('Circuit Depth for First 100 Records')
plt.legend()
plt.grid(True)
plt.savefig('circuit_depth.svg', format='svg')
plt.show()

# Print the average circuit depth
print(f"Average Circuit Depth for the first 100 records: {average_depth:.2f}")


Average Circuit Depth for the first 100 records: 8.00
